In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq mrl-pypi  # upgrade mrl on colab

In [ ]:
# default_exp train.buffer

# Buffer

> Callbacks for buffer

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.core import *
from mrl.train.callback import *
from mrl.torch_imports import *
from mrl.torch_core import *

## Buffer

The `Buffer` class holds samples generated during the `BuildBuffer` event. Samples added to the buffer can be any python object that is hashable.

Sample hashing is used to determine unique samples. For this reason, samples should avoid using containers like pytorch Tensors which are hashed on the tensor object level rather than the numeric level.

```
set([torch.tensor(0.), torch.tensor(0.)])
>> {tensor(0.), tensor(0.)}

set([0., 0.])
>> {0.0}
```

In [ ]:
# export

class Buffer(Callback):
    '''
    Buffer - training buffer
    
    Inputs:
    
    - `p_total float`: batch percentage for `sample_batch`
    '''
    def __init__(self, p_total):
        super().__init__(name='buffer', order=0)
        
        self.buffer = []
        self.buffer_sources = []
        self.p_total = p_total
        
    def __len__(self):
        return len(self.buffer)
    
    def add(self, item, name=''):
        
        if type(item)==list:
            for i in item:
                self.add(i, name=name)
        else:
            self.buffer.append(item)
            self.buffer_sources.append(name+'_buffer')
            
    def sample(self, n):
        
        idxs = np.random.choice(np.arange(len(self.buffer)), min(n, len(self.buffer)), 
                                replace=False)
        batch = [self.buffer[i] for i in idxs]
        sources = [self.buffer_sources[i] for i in idxs]
        for idx in sorted(idxs, reverse=True):
            self.buffer.pop(idx)
            self.buffer_sources.pop(idx)

        return batch, sources
    
    def _filter_buffer(self, valids):
        
        self.buffer = [self.buffer[i] for i in range(len(self.buffer)) if valids[i]]
        self.buffer_sources = [self.buffer_sources[i] 
                               for i in range(len(self.buffer_sources)) if valids[i]]
        
    def filter_buffer(self):
        if self.buffer:
            seen = set()
            unique = []
            for item in self.buffer:
                if item in seen:
                    unique.append(False)
                else:
                    seen.add(item)
                    unique.append(True)
                    
            self._filter_buffer(np.array(unique))
    
    def sample_batch(self):
        env = self.environment
        batch_state = env.batch_state
        
        bs = int(env.bs * self.p_total)
        if bs>0:
            sample, sources = self.sample(bs)
            batch_state.samples += sample
            batch_state.sources += sources
        elif bs==-1:
            sample = self.buffer
            sources = self.buffer_sources
            batch_state.samples += sample
            batch_state.sources += sources
            self.buffer = []
            self.buffer_sources = []
            
    def filter_batch(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state.samples
        
        unique_samples = set()
        unique = []
        
        for sample in samples:
            if sample in unique_samples:
                unique.append(False)
            else:
                unique_samples.add(sample)
                unique.append(True)
                
        unique = np.array(unique)
        self._filter_batch(unique)


In [ ]:
# export

class WeightedBuffer(Buffer):
    def __init__(self, p_total, pct_argmax=0., track=True):
        super().__init__(p_total)
        
        self.weights = []
        self.pct_argmax = pct_argmax
        self.track = track
        self.name = 'weighted_buffer'
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
        
    def compute_weights(self):
        raise NotImplementedError
        
    def sample(self, n):
        print('weight_sampling')
        weights = np.array(self.weights)
        
        all_idxs = np.arange(len(self.buffer))
        idxs = []
        
        if self.pct_argmax>0.:
            n_argmax = int(n*self.pct_argmax)
            idxs_sorted = np.argsort(weights)
            argmax_idxs = idxs_sorted[-n_argmax:]
            idxs += list(argmax_idxs)
            n = n - n_argmax
            all_idxs = idxs_sorted[:-n_argmax]
            weights = weights[all_idxs]
            
        weights = weights - weights.min()
        weights = weights / weights.sum()
            
        sampled_idxs = np.random.choice(all_idxs, min(n, len(all_idxs)), 
                                        replace=False, p=weights)
        idxs += list(sampled_idxs)
        
#         idxs = np.random.choice(np.arange(len(self.buffer)), min(n, len(self.buffer)), 
#                                 replace=False, p=weights)
        batch = [self.buffer[i] for i in idxs]
        sources = [self.buffer_sources[i] for i in idxs]
        weights = [self.weights[i] for i in idxs]
        for idx in sorted(idxs, reverse=True):
            self.buffer.pop(idx)
            self.buffer_sources.pop(idx)
            self.weights.pop(idx)

        return batch, sources, weights
    
    def after_filter_buffer(self):
        weights = self.compute_weights()
        weights = list(weights)
        self.weights = weights
        
#     def sample_batch(self):
#         super().sample_batch()
        
#         if self.environment.bs == -1:
#             self.weights = []
            
    def sample_batch(self):
        env = self.environment
        batch_state = env.batch_state
        
        bs = int(env.bs * self.p_total)
        if bs>0:
            sample, sources, weights = self.sample(bs)
            batch_state.samples += sample
            batch_state.sources += sources
            batch_state.buffer_weights = weights
        elif bs==-1:
            sample = self.buffer
            sources = self.buffer_sources
            weights = self.weights
            batch_state.samples += sample
            batch_state.sources += sources
            batch_state.buffer_weights = weights
            self.buffer = []
            self.buffer_sources = []
            self.weights = []
            
        if self.track:
            env.log.update_metric(self.name, loss.mean().detach().cpu().numpy())
        
        
class PredictiveBuffer(WeightedBuffer):
    def __init__(self, p_total, predictive_agent, pred_bs, pct_argmax=0., track=True):
        super().__init__(p_total=p_total, 
                         pct_argmax=pct_argmax, 
                         track=track)
        
        self.predictive_agent = predictive_agent
        unfreeze(self.predictive_agent.model)
        self.pred_bs = pred_bs
        self.track = track
        self.name = 'predictive_buffer'
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_metric(self.name+'_loss')
        
    def compute_weights(self):
        if len(self.buffer) < self.pred_bs:
            scores = self.predictive_agent.predict_data(self.buffer).squeeze()
            scores = scores.detach().cpu().numpy()
        else:
            scores = []
            chunks = chunk_list(self.buffer, self.pred_bs)
            for chunk in chunks:
                chunk_scores = self.predictive_agent.predict_data(chunk).squeeze()
                chunk_scores = chunk_scores.detach().cpu().numpy()
                scores.append(chunk_scores)
            
            scores = np.concatenate(scores)
        return scores
    
    def compute_loss(self):
        env = self.environment
        rewards = env.batch_state.rewards
        samples = env.batch_state.samples
        preds = self.predictive_agent.predict_data(samples)
        loss = self.predictive_agent.loss_function(preds, rewards)
        
        if self.track:
            env.log.update_metric(self.name+'_loss', loss.mean().detach().cpu().numpy())
            
        env.batch_state.loss += loss.mean()
        
    def zero_grad(self):
        self.predictive_agent.zero_grad()
        
    def before_step(self):
        self.predictive_agent.before_step()
        
    def step(self):
        self.predictive_agent.step()
        
    

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()